In [ ]:
!pip install --upgrade google-genai

In [49]:
import json
from google.cloud import bigquery

def vector_search(user_question):
  client = bigquery.Client()

  sql = """
  SELECT query.query, base.question, base.answer, distance
  FROM VECTOR_SEARCH(
  TABLE `qwiklabs-gcp-01-3005c83a3aae.faq.aurora_faq_with_embeddings`, 'ml_generate_embedding_result',
  (
  SELECT text_embedding, content AS query
  FROM ML.GENERATE_TEXT_EMBEDDING(
  MODEL `qwiklabs-gcp-01-3005c83a3aae.faq.embedding_model`,
  (SELECT @user_question AS content))
  ),
  top_k => 5, options => '{"fraction_lists_to_search": 0.01}')
  """

  job_config = bigquery.QueryJobConfig(
      query_parameters=[
          bigquery.ScalarQueryParameter("user_question", "STRING", user_question),
          # Options must be passed as a JSON string
          bigquery.ScalarQueryParameter("search_options", "JSON", json.dumps({"fraction_lists_to_search": 0.01}))
      ]
  )

  try:
    # Start the query job.
    # print(f"Executing vector search for: '{user_question}'")
    query_job = client.query(sql, job_config=job_config)

    # Wait for the job to complete and fetch results.
    results = query_job.result() # Waits for job completion

    # Process results into a more usable format (list of dictionaries)
    output_rows = []
    for row in results:
        output_rows.append({
            "query": row.query,
            "question": row.question,
            "answer": row.answer,
            # "distance": row.distance
        })

    # print(f"Found {len(output_rows)} similar questions.")
    return output_rows

  except Exception as e:
      print(f"An error occurred during the BigQuery vector search: {e}")
      return []


# print(vector_search("aurora"))
result = vector_search("Can I get help with taxes")

In [52]:
def format_prompt(user_question, results):
  output = f"User Query: {user_question} \n\nUse the relevant information below to help the user:"
  for result in results:
    output = output + "\n\n" + "Question: " + result.get("question") + "\n" + "Answer: " +result.get("answer")

  return output

print(format_prompt("Can I get help with taxes", result))

User Query: Can I get help with taxes 

Use the relevant information below to help the user:

Question: What is the local tax rate for businesses?
Answer: Business taxes include a 4% sales tax and a 2% local business tax on net profits, in addition to any applicable state taxes.

Question: Who do I contact about property tax assessments?
Answer: For property tax assessments, contact the Aurora Bay Assessors’ Office at Town Hall, or call (907) 555-0145 for assistance.

Question: question
Answer: answer

Question: How can I volunteer in community events?
Answer: You can sign up through the Community Center’s volunteer portal or attend volunteer meetings announced on the town’s website and local bulletin boards.

Question: How do I request a building permit?
Answer: Building permit applications can be obtained at the Town Hall’s Planning & Development Office or on the official website. A site inspection is required before approval.


In [54]:
from google import genai
from google.genai import types
import base64



def response_model(user_input):
  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-01-3005c83a3aae",
      location="us-central1",
  )

  si_text1 = """You a helpful chatbot for people asking common questions about Aurora. Only respond to questions about Aurora."""

  # model = "gemini-2.0-flash-lite-001"
  model = "gemini-2.5-pro-preview-03-25"
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text=user_input)
      ]
    ),
  ]
  generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    seed = 0,
    max_output_tokens = 1346,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )],
    system_instruction=[types.Part.from_text(text=si_text1)],
  )

  response = client.models.generate_content(
          model=model,
          contents=contents,
          config = generate_content_config,
        )
  # print("response:",response.text)
  return response.text


def main(user_input):
  # runs vector search for closes 5 embeddings
  search_results = vector_search(user_input)

  # formats the prompt with user question and 5 FAQs
  formatted_prompt = format_prompt(user_input, search_results)
  # print(formatted_prompt)

  # sends formated prompt to gemini to construct a response to the answer
  run_response = response_model(formatted_prompt)

  return run_response

#user question
user_input = "can I get some help with taxes"


print(f"Question: {user_input}\n--------------------------------\n\n")
print(main(user_input))


Question: can I get some help with taxes
--------------------------------


Based on the information available for Aurora:

*   If you're asking about **business taxes**, Aurora has a 4% sales tax and a 2% local business tax on net profits, in addition to any applicable state taxes.
*   If your question is about **property taxes**, you should contact the Aurora Bay Assessors’ Office at Town Hall, or call (907) 555-0145 for assistance with assessments.

If you have a different tax question about Aurora, please provide more details!
